# #60 RC Design checking of Tee beam
<i>Creates a multispan Tee beam with reinforcement ready for RC Design checks</i>
***

![Preview](../_img/jupyter_notebook_060.png)

In [ ]:
# Input Params
spans_lengths = [5, 10, 5] #m
tee_section_width = 1.0
tee_section_depth = 1.0
tee_section_thk_flange = 0.2
tee_section_thk_web = 0.2

loading = 100

#### Connect to LUSAS and create a new model database

In [ ]:
import sys; sys.path.append('../') # Reference modules in parent directory
from shared.LPI_21_1 import *
lusas = get_lusas_modeller()
if lusas.existsDatabase() and lusas.db().isModified():
    raise Exception("This script will create a new model. Please save or close the current model and try again")

# Create a new model
lusas.newProject("Structural", "RC Beam Design.mdl")
# Get a reference to the model database
db = lusas.getDatabase()
# Set the analysis category & vertical axis
db.setAnalysisCategory("3D")
db.setVerticalDir("Z")
# Set the unit system
db.setModelUnits("kN,m,t,s,C")

#### Create model attributes

Create the mesh attribute:

In [ ]:
''' Create a mesh attribute'''
# Beam mesh with BMI21 (2 Noded Linear) Elements at 1m lengths
mesh_attr = db.createMeshLine("Beam Mesh").setSize("BMI21", 1)

Create the geometric attribute using a parametric section:

In [ ]:
''' Create a geometric attribute'''
# Create a parametric section utility
util = db.createParametricSection('Tee Section Utility')
util.setType("T")
util.setDimensions(['B', 'D', 'tf', 'tw', 'r'], [tee_section_width, tee_section_depth, tee_section_thk_flange, tee_section_thk_web, 0.0])

# Now create the attribute which is based on the utility but provides additional transformations such as eccentricity and tapering
geom_section_attr = db.createGeometricLine('Tee Section')
geom_section_attr.setFromLibrary("Utilities", "", 'Tee Section Utility', 0, 0, 0)

Create a function for easier rebar definition:

In [ ]:
def add_rebar(attr:IFReinforcementSection, row:int, faceIndex:int, layerIndex:int, barsCount:int, start:int, end:int, gap:float, dia:float):
    attr.setReinforcementValue("rebar", row, "faceIndex", faceIndex)
    attr.setReinforcementValue("rebar", row, "layerIndex", layerIndex)
    attr.setReinforcementValue("rebar", row, "barsCount", barsCount)
    attr.setReinforcementValue("rebar", row, "start", start)
    attr.setReinforcementValue("rebar", row, "end", end)
    attr.setReinforcementValue("rebar", row, "gap", gap)
    attr.setReinforcementValue("rebar", row, "barsDiameter", dia)
    attr.setReinforcementValue("rebar", row, "altBarsDiameter", -1.0)
    attr.setReinforcementValue("rebar", row, "endBarsDiameter", -1.0)
    attr.setReinforcementValue("rebar", row, "materialRef", "R1")

Create the reinforcement attribute:

In [ ]:
''' Create a Reinforcement attribute'''
attr = db.createReinforcementSection("RnfSct1")
attr.setGeometricAttribute(geom_section_attr)
#attr.setValue("calculateCrackWidths", True)
attr.setReinforcementValue("face", 0, "faceIndex", "All")
attr.setReinforcementValue("face", 0, "actualCover", 0.03)
attr.setReinforcementValue("face", 0, "linkAllowance", 0.0)
attr.setReinforcementValue("face", 0, "allowableCrackWidth", 0.0)
attr.setReinforcementValue("face", 0, "nominalCover", -1.0)

add_rebar(attr, row=0, faceIndex=1, layerIndex=1, barsCount=3, start=-1, end=-1, gap=0.0, dia=0.025)
add_rebar(attr, row=1, faceIndex=1, layerIndex=2, barsCount=2, start=-1, end=-1, gap=0.0, dia=0.025)
add_rebar(attr, row=2, faceIndex=5, layerIndex=1, barsCount=10, start=-1, end=-1, gap=0.0, dia=0.025)


rebar_line_attr = db.createReinforcementLine("RnfLn2")
rebar_line_attr.setSegmentValue("Longitudinal", 0, "reinforcementSection", "RnfSct1")
rebar_line_attr.setSegmentValue("Longitudinal", 0, "stretch", True)

# Link the geometric attribute with the associated reinforcement
geom_section_attr.setReinforcement(rebar_line_attr)


In [ ]:
attr = db.createIsotropicMaterial("Iso1", 35.0E6, 0.2, 2.54842, 10.0E-6)
attr.setDefinitionMenuID(1, "None" , True)
attr.setDescription("C40/50 | Concrete | EN1992-1-1:2004/2014")
attr.createValue("Grade", 0, 0, 0, 0, 0, 0, 0)
attr.setValue("Grade", "C40/50")
attr.createValue("Region", 0, 0, 0, 0, 0, 0, 0)
attr.setValue("Region", "Europe")
attr.createValue("Standard", 0, 0, 0, 0, 0, 0, 0)
attr.setValue("Standard", "EN1992-1-1:2004/2014")
attr.createValue("Material", 0, 0, 0, 0, 0, 0, 0)
attr.setValue("Material", "Concrete")
attr.createValue("ec3", 0, 0, 0, 0, 0, 0, 0)
attr.setValueDescription("ec3", "Compressive strain limit 3", False)
attr.setValue("ec3", 1.75E-3)
attr.createValue("fcd3", 0, 1, -2, 0, 0, 0, 0)
attr.setValueDescription("fcd3", "Design compressive strength 3", False)
attr.setValue("fcd3", 26.66666667E3)
attr.createValue("fck", 0, 1, -2, 0, 0, 0, 0)
attr.setValueDescription("fck", "Characteristic compressive cylinder strength of concrete at 28 days", False)
attr.setValue("fck", 40.0E3)
attr.createValue("fctm", 0, 1, -2, 0, 0, 0, 0)
attr.setValueDescription("fctm", "Mean value of axial tensile strength of concrete", False)
attr.setValue("fctm", 3.5E3)
attr.createValue("ecu3", 0, 0, 0, 0, 0, 0, 0)
attr.setValueDescription("ecu3", "Ultimate compressive strain limit 3", False)
attr.setValue("ecu3", 3.5E-3)
attr.createValue("E3", 0, 1, -2, 0, 0, 0, 0)
attr.setValueDescription("E3", "Modulus of elasticity 3", False)
attr.setValue("E3", 15.238095238E6)


attr = db.createIsotropicMaterial("Iso2", 200.0E6, 0.3, 7.85, 12.0E-6)
attr.setDefinitionMenuID(1, None , True)
attr.setDescription("500B | Steel - Reinforcing bar | EN1992-1-1:2004/2014")
attr.createValue("Region", 0, 0, 0, 0, 0, 0, 0)
attr.setValue("Region", "Europe")
attr.createValue("Standard", 0, 0, 0, 0, 0, 0, 0)
attr.setValue("Standard", "EN1992-1-1:2004/2014")
attr.createValue("Grade", 0, 0, 0, 0, 0, 0, 0)
attr.setValue("Grade", "500B")
attr.createValue("Material", 0, 0, 0, 0, 0, 0, 0)
attr.setValue("Material", "Steel - Reinforcing bar")
attr.createValue("fy", 0, 1, -2, 0, 0, 0, 0)
attr.setValueDescription("fy", "Characteristic yield strength of reinforcement", False)
attr.setValue("fy", 500.0E3)
attr.createValue("epu", 0, 0, 0, 0, 0, 0, 0)
attr.setValueDescription("epu", "Characteristic ultimate strain limit", False)
attr.setValue("epu", 0.05)


material_attr = db.createCompoundMaterial("Mat3").setReinforcedConcrete()
material = db.getAttribute("Isotropic Material", "Iso1")
material_attr.addMaterial(material, "S1")
material = db.getAttribute("Isotropic Material", "Iso2")
material_attr.addMaterial(material, "R1")
material_attr.setActiveMaterial(0)
material_attr.setDefinitionMenuID(214, None , True)
material_attr.setDescription("EN1992")
material_attr.createValue("Design code", 0, 0, 0, 0, 0, 0, 0)
material_attr.setValue("Design code", "EN1992")

In [ ]:
''' Create a support attribute '''
fixed_support_attr = db.createSupportStructural("Fixed")
fixed_support_attr.setStructural("R", "R", "R", "F", "F", "F", "F", "F", "F")

''' Create a support attribute '''
pinned_support_attr = db.createSupportStructural("Pinned")
pinned_support_attr.setStructural("F", "R", "R", "F", "F", "F", "F", "F", "F")

#### Create the model geometry

In [ ]:
''' Create deck lines'''
deck_lines:list[IFLine] = []

cur_x = 0
for length in spans_lengths:
    # Get the geometry data object
    geomData = lusas.geometryData().setAllDefaults()
    # set the options for creating points from coordinates
    geomData.setLowerOrderGeometryType("coordinates")
    # Set the coordinates of the first point
    geomData.addCoords(cur_x, 0, 0)
    # Set the coordinates of the second point
    geomData.addCoords(cur_x + length, 0, 0)
    # Create the line object from the geometry data
    objs = db.createLine(geomData) # Modeller returns an object set containing all the lines created, in this case we only have 1.
    # Get the line
    deck_lines.append(win32.CastTo(objs.getObject("Lines"), "IFLine"))
    # Increment the current x position
    cur_x += length


#### Assign attributes to the model geometry

In [ ]:
''' Assign the attributes to the deck_lines '''
# get the assignment object
assignment = lusas.assignment().setAllDefaults()
# Assign the mesh
mesh_attr.assignTo(deck_lines, assignment)
# Assign the geometry
geom_section_attr.assignTo(deck_lines, assignment)
# Assign the material
material_attr.assignTo(deck_lines, assignment)

''' Assign the supports to the points of the line '''
# Assign the fixed support to the first point
fixed_support_attr.assignTo(deck_lines[0].getStartPoint(), assignment)
# Assign the pinned support to the remaining points
pinned_support_attr.assignTo([p.getEndPoint() for p in deck_lines], assignment)

In [ ]:
db.updateMesh()

#### Create load cases and load attributes and assign the attributes to the model geometry

In [ ]:
# Get the automatically created loadcase in analysis 1 and add automatic gravity to it
# NOTE: getLoadset and setName function returns a reference to the IFLoadset baseclass and must be cast to IFLoadcase to access the addGravity function
gravity_loadcase : IFLoadcase = win32.CastTo(db.getLoadset("Loadcase 1", 0).setName("Gravity"), "IFLoadcase")
gravity_loadcase.addGravity(True)

In [ ]:
# Create a beam load attribute
uniform_load_attr = db.createLoadingBeamDistributed("UDL")
uniform_load_attr.setBeamDistributed("Parametric", "Global", "beam")
uniform_load_attr.addRow(0.0, 0.0, 0.0, -loading, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, -loading, 0.0, 0.0, 0.0)

# Create a loadcase for loading to be applied to each span
live_load_loadcases = [db.createLoadcase(f"Live Load {i+1}") for i in range(len(deck_lines))]

# Assign live loads to each span
for i, loadcase in enumerate(live_load_loadcases):
    assign = lusas.assignment().setAllDefaults().setLoadset(loadcase)
    uniform_load_attr.assignTo(deck_lines[i], assign)

#### Create design combinations

In [ ]:
# Smart Combination for permanent effects
combination_qp = db.createCombinationSmart("SLS QP")
combination_qp.addEntry(1,1, gravity_loadcase)
for loadcase in live_load_loadcases:
    combination_qp.addEntry(0,0.5,loadcase)

In [ ]:
# Smart Combination for service effects
combination_char = db.createCombinationSmart("SLS Char")
combination_char.addEntry(1,1, gravity_loadcase)
for loadcase in live_load_loadcases:
    combination_char.addEntry(0,1.0,loadcase)

In [ ]:
# Smart Combination for Ultimate effects
combination_uls = db.createCombinationSmart("ULS")
combination_uls.addEntry(1,1, gravity_loadcase)
for loadcase in live_load_loadcases:
    combination_uls.addEntry(0,1.5,loadcase)

<H2>Solving the Analysis</H2>

In [ ]:
db.getAnalysis("Analysis 1").solve(True)
db.openAllResults(False)

In [ ]:
lusas.view().insertDiagramsLayer()
lusas.view().diagrams().setResultsTransformNone()
lusas.view().diagrams().setResults("Force/Moment - Thick 3D Beam", "My")
lusas.view().setFromAxis("+X+Z")

<div class="alert alert-block alert-info">
<b>Note:</b> The RC Design results can now be obtained using the Modeller user interface as follows:
<dl>
<i><b>Set the design code</i></b>
<dd><b>1.</b> Design > RC Frame Design</dd>
<i><b>Specify and assign member design attributes</i></b>
<dd><b>2.</b> Attributes > Design > RC Frame Design</dd>
<i><b>Create a design results loadset which carries out design checks</i></b>
<dd><b>3.</b> Design > RC Frame Design Results</dd>
<dl>
</div>